# <center> Практические задания по цифровой обработке сигналов </center>
# <center> Первая лабораторная работа </center>


В данной работе Вы познакомитесь с основными методами работы с аудиоданными в Python. Разбересь в том, как работает свертка, и примените пару интересных фильтров.

# Задание 1. Работа с аудиофайлами в Python (1 балл)

## Теория

Звук - это аналоговый сигнал. То есть он является непрерывным по времени и по значениям. Для того, чтобы работать со звуком на цифровом устройстве, надо преобразовать его в цифровое представление. Для этого надо разделить непрерывный сигнал на промежутки времени (дискретизация сигнала) и разбить непрерывные значения на интервалы (квантование сигнала). Выбраные параметры дискретизации и квантования сигнала напрямую влияют на качество цифрового сигнала. 

<!-- 
## Практика

1. Что хранится в .wav файле? Как узнать параметры дискретизации и квантования .wav файла? 

2. Запишите аудиофайл со своим голосом. Загрузите его. Попробуйте поменять ему частоту дискретизации. Нарисуйте форму волны считанного файла. Воспроизведите полученные сигналы. При какой частоте дискретизации становится невозможно разобрать человеческую речь?   

3. Чем .wav отличается от других кодеков, например .mp3 или .ogg? -->


<!-- ### Подсказка

Записать цифровой сигнал можно при помощи, например, [Audacity](https://www.audacityteam.org) или [Adobe Audition](https://www.adobe.com/ru/products/audition.html). Для считывания файлов воспользуйтесь библиотекой [scipy](https://www.scipy.org) или [librosa](https://librosa.org/doc/latest/index.html). Для воспроизведения аудиофайла удобно использовать класс Audio из модуля IPython.display, а для отрисовки - matplotlib. -->

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

### 1. Что хранится в .wav файле? Как узнать параметры дискретизации и квантования .wav файла?

In [ ]:
# .wav файл содежит две области: заголовок и аудиоданные. В заголовке хранится количество каналов, параметры дискретизации и квантования, а также некоторые другие данные.

# Узнать частоту сэмплирования и битность можно, например, так:

import wave

with wave.open('resources/lab1_16bits.wav') as f:
    print(f.getparams())

# Функция возвращает аналог named_tuple, в котором 'sampwidth' - количество байт на сэмпл, "framerate" - собственно, фреймрейт.

### 2. Запишите аудиофайл со своим голосом. Загрузите его. Попробуйте поменять ему частоту дискретизации. Нарисуйте форму волны считанного файла. Воспроизведите полученные сигналы. При какой частоте дискретизации становится невозможно разобрать человеческую речь?

**Подсказка**

Записать цифровой сигнал можно при помощи, например, [Audacity](https://www.audacityteam.org) или [Adobe Audition](https://www.adobe.com/ru/products/audition.html). Для считывания файлов воспользуйтесь библиотекой [scipy](https://www.scipy.org) или [librosa](https://librosa.org/doc/latest/index.html). Для воспроизведения аудиофайла удобно использовать класс Audio из модуля IPython.display, а для отрисовки - matplotlib.

In [ ]:
# передискретизация
import numpy as np

desirable_SR = 20000  # желаемая частота дискретизации

with wave.open('resources/lab1_my_voice.wav') as f:
    with wave.open('resources/lab1_my_voice_processed.wav', 'wb') as g:
        data = f.readframes(-1)

        PARAMS = (2, 2, desirable_SR, f.getnframes(), 'NONE', 'not compressed')
        g.setparams(PARAMS)
        g.writeframes(data)

        Stereo = np.reshape(np.frombuffer(data, np.int16),
                            (-1, 2))  # левый канал - в первом столбце, правый - во втором
        Left, Right = Stereo[:, 0], Stereo[:, 1]

In [ ]:
# Вопроизведение
from IPython.display import Audio
Audio([Left, Right], rate=desirable_SR, autoplay=True)
# При частоте дискретизации в 20000 мою речь невозможно разобрать

In [ ]:
# График
plt.plot(Left)
plt.show()

### 3. Чем .wav отличается от других кодеков, например .mp3 или .ogg?

In [ ]:
# .wav файл хранит несжатые данные

# Задание 2. Гармонические сигналы (1 балл)

## Теория
[Гармонические колебания](https://ru.wikipedia.org/wiki/Гармонические_колебания) -  колебания, при которых физическая величина изменяется с течением времени по гармоническому (синусоидальному/косинусоидальному) закону. 

В общем случае гармонические колебания задаются формулой:

$$y=A \cos(\omega t+\varphi_0)$$

где $А$ - это амплитуда, $\omega$ – циклическая частота (радиан/с), $\varphi$ - фаза (сдвиг), $t$ – время. 


In [ ]:
# Сначала определим функцию для отрисовки сигнала с хорошим масштабом и сеткой
# Это поможет легче анализировать сигнал
def draw_signal(data, figsize=(14, 14)):
    plt.figure(figsize=figsize)
    plt.plot(data, linewidth=2)
    plt.minorticks_on()
    plt.xticks(np.arange(0, 1000, step=100))
    plt.yticks(np.arange((data.min().round()) // 10 * 10,
                         (data.max().round()) // 10 * 10 + 10, step=5))
    plt.grid(which='major',
             color='k',
             linewidth=1)
    plt.grid(which='minor',
             color='k',
             linestyle=':')
    plt.show()

In [ ]:
# Читаем данные с подготовленными сигналами
import pickle

with open("resources/data.pickle", "rb") as f:
    test_data = pickle.load(f)
# Теперь можно приступать к практике!

## Практика

Постройте графики трех сигналов a, b и c из test_data['task2']. Попробуйте подобрать коэффициенты для этих сигналов. Сгенерируйте сигналы (1000 отсчетов) с подобранными коэффициентами. Постройте графики сгенерированных сигналов и пройдите тест на схожесть с оригинальным.


Подсказка. Фаза, период и амплитуда сигнала - целочисленные. Для генерации пользуйтесь библиотекой numpy и функциями arange, sin, cos.

### Сигнал ***a***

In [ ]:
draw_signal(test_data['task2']['a'])

In [ ]:
amp = 45
T = 425
freq = 2 * np.pi / T
phase = 0

t = np.arange(0, 1000)
a = amp * np.cos(t * freq + phase)

In [ ]:
draw_signal(a)

In [ ]:
assert len(a) == 1000
assert np.allclose(a, test_data["task2"]["a"], atol=1)
print("Ok!")

**Подобранные коэффициенты для сигнала 'a':**

1. Амплитуда - $A= 45 $

2. Угловая частота ($ \displaystyle\omega =\frac{2\pi}{T}) = \frac{2\pi}{425}$

3. Фаза - $\phi= 0 $

### Сигнал ***b***

In [ ]:
draw_signal(test_data['task2']['b'])

In [ ]:
amp = 50
T = 100
freq = 2 * np.pi / T
phase = 0

t = np.arange(0, 1000)
b = amp * np.sin(t * freq + phase)

In [ ]:
draw_signal(b)

In [ ]:
assert len(b) == 1000
assert np.allclose(b, test_data["task2"]["b"], atol=1)
print("Ok!")

**Подобранные коэффициенты для сигнала 'b':**

1. Амплитуда - $A= 50$

2. Угловая частота ($ \displaystyle\omega =\frac{2\pi}{T}) = \frac{2\pi}{100}$

3. Фаза - $\phi= 0 $ (использован синус; 3pi/2, если использовать косинус.
Если необходимо целое число, то прибавляем 2pi к 3pi/2 до тех пор, пока не получим число, достаточно близкое к целому. Например, 58250335

### Сигнал ***c***

In [ ]:
draw_signal(test_data['task2']['c'])

In [ ]:
# сигнал состоит из двух гармоник
t = np.arange(0, 1000)

amp1 = -50
T1 = 100
freq1 = 2 * np.pi / T1
phase1 = 0
first = amp1 * np.sin(t * freq1 + phase1)

amp2 = 45
T2 = 425
freq2 = 2 * np.pi / T2
phase2 = 0
second = amp2 * np.cos(t * freq2 + phase2)

c = first + second
draw_signal(c)

In [ ]:
assert len(c) == 1000
assert np.allclose(c, test_data["task2"]["c"], atol=1)
print("Ok!")

# Задание 3. Свертка (3 балла)

## Теория
Одна из наиболее частых операций, которая выполняется при обработке сигналов, это свёртка. Свёртка имеет много различных применений, например, с ее помощью можно убрать из сигнала шумы или применить к сигналу эффект эхо.


Свёртка — это математическая операция, применённая к двум функциям f и g и порождающая третью функцию. Операцию свёртки можно интерпретировать как «схожесть» одной функции с отражённой и сдвинутой копией другой.  Другими словами, преобразование свёртки однозначно определяет выходной сигнал y(t) для установленного значения входного сигнала x(t) при известном значении функции импульсного отклика системы h(t).

<!-- ![Convolution](resources/Convolution_of_box_signal_with_itself2.gif "Convolution") -->
![Convolution](resources/Convolution_of_box_signal_with_itself2.gif "Convolution")

Формула свёртки:
$$y_t=\frac{1}{2} \int_0^T x(\tau)h(t-τ)dτ$$
где $\tau$  - длительность импульсной переходной характеристики.

## Практика
Реализуйте операцию свёртки. Сравните её с существующей реализацией scipy.signal.convolve. Постройте графики фильтра, исходного сигнала и результата свертки.

In [ ]:
def m(arr1, arr2):
    assert len(arr1) == len(arr2)
    res = 0
    for x, y in zip(arr1, arr2):
        res += x * y

    return res


def convolve(in1, in2):
    result = []
    l1 = len(in1)
    l2 = len(in2)

    np.flip(in2)

    i = 0
    while i < l1 + l2 - 1:
        start1 = i - l2 + 1 if i >= l2 else 0
        end1 = i + 1 if i < l1 else l1
        start2 = l2 - i - 1 if i <= l2 - 1 else 0
        end2 = l2 + l1 - i - 1 if i >= l1 else l2
        result.append(m(in1[start1:end1], in2[start2:end2]))
        i += 1

    return result

In [ ]:
import scipy

def test_convolve(a, b, print_debug=False):
    my_result = convolve(a, b)
    scipy_result = scipy.signal.convolve(a, b, method='direct')
    if print_debug:
        print(f"Your result {my_result}")
        print(f"Scipy result {scipy_result}")
    assert np.allclose(my_result, scipy_result), f"Test {a} conv {b} failed"
    print("Ok!")

In [ ]:
a = np.repeat([0, 1, 0], 10)
b = np.array([0, 1, 2, 3, 2, 1, 0])

In [ ]:
test_convolve(a, b, print_debug=False)

### Нарисуйте результат свертки a и b

Сигнал **а**

In [ ]:
plt.plot(a)
plt.show()

Сигнал **b**

In [ ]:
plt.plot(b)
plt.show()

Свертка **a** и **b**

In [ ]:
plt.plot(convolve(a, b))
plt.show()

Все 3 графика на одной картинке:

In [ ]:
c = convolve(a, b)
x1 = np.arange(0, len(a))
x2 = np.arange(0, len(b))
x3 = np.arange(0, len(c))

plt.plot(x1, a, '-', x2, b, '--', x3, c, '-.')
plt.show()

# Задание 4. * Алгоритм Карплуса-Стронга 

Реализуйте  [Алгоритм Карплуса-Стронга](https://en.wikipedia.org/wiki/Karplus%E2%80%93Strong_string_synthesis). В качестве фильтра используйте усреднитель двух смежных отсчетов. Проверьте результат. 

Отрисуйте и воспроизведите полученный сигнал. На что влияют параметры генерации? Попробуйте имитировать звучание разных струн гитары.

In [ ]:
def lowpass_filter(a):
    res = np.array([a[0]])
    for i in range(1, len(a)):
        res = np.append(res, (a[i] + a[i - 1]) // 2)
    return res

def delay(data, N):
    if len(data) > N:
        return data
    else:
        return delay(np.append(data, lowpass_filter(data)), N)

def karplus_strong(noise, N):
    return delay(noise, N)[:N]

In [ ]:
np.random.seed(seed=1)
sample_rate = 44100
frequency = 82.41
sec = 2
gen_len = sample_rate * sec
noise = (2 * np.random.uniform(-1, 1, int(sample_rate / frequency)))  # [-1, 1]

gen_wav = karplus_strong(noise, gen_len)

assert np.allclose(gen_wav[:len(noise)], noise), "Generated signal must starting with noise"
assert np.allclose(gen_wav[len(noise)], (noise[0]) / 2), "Out of range samples eq 0."
assert np.allclose(gen_wav[len(noise) + 1: 2 * len(noise)],
                   (noise[:-1] + noise[1:]) / 2), "Bad requrent rule( 1 iteration)"
assert np.allclose(gen_wav[2 * len(noise)], (noise[0] / 2 + noise[-1]) / 2), "Bad requrent rule( 2 iteration)"
assert np.allclose(gen_wav[2 * len(noise) + 2: 3 * len(noise)], (
        ((noise[:-1] + noise[1:]) / 2)[:-1] + ((noise[:-1] + noise[1:]) / 2)[
                                              1:]) / 2), "Bad requrent rule( 3 iteration)"
print('All Ok!')

### Попробуем покрутить параметры генерации

Сгенерируем гитарные ноты:

In [ ]:
# YOUR CODE HERE

Визуализируем затухание амплитуды:

In [ ]:
plt.figure(figsize=(10, 5))
plt.xlabel('n', fontsize=14)
plt.ylabel('Амплитуда', fontsize=14)
plt.xlim(0, gen_len)
plt.plot(np.linspace(0, gen_len + 1, gen_len), gen_wav)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.grid()

Визуализируйте затухание амплитуд гитарных нот:

In [ ]:
# YOUR CODE HERE